# Gather
Read in campaign finance dataset

### Table Structure

**Raw data structure**

* Candidate master
    * one record for each candidate
* Contributions to Candidate
    * one record per contribution, including committe (PAC) id and candidate id

Contributions to Candidate -- match on candidate id -->
Candidate master

**Master table** 
* one row per contribution including committee id, contribution value (\$), PAC attributes, candidate id, candidate state, candidate party

**Tidy dataset for analysis (to do in R)** 
* one row per candidate
* one column per PAC with value of contribution (\$)
* one summary column with total value of contributions
* one column for state
* one column for party


In [272]:
import pandas as pd
import numpy as np
import requests
import sqlalchemy
import datetime as dt
import random

In [273]:
# Open text file containing candidate dataset
df_list = []

with open(file='cn.txt',encoding='utf-8') as file:
    for row in file:
        df_list.append(file.readline()[:-1].split(sep="|"))        

# Transform to Pandas data frame      
cand = pd.DataFrame(df_list)

# Add column headers from data dictionary
cand_cols = requests.get('http://classic.fec.gov/finance/disclosure/metadata/cn_header_file.csv')
cand_cols = str(cand_cols.content[:-2]).split(sep=',')
cand_cols[0] = 'CAND_ID'

cand.columns=cand_cols

# Visual inspection of data
cand.head()

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP'
0,H0AL02087,"ROBY, MARTHA",REP,2014,AL,H,02,I,C,C00462143,3260 BANKHEAD AVE,,MONTGOMERY,AL,361062448
1,H0AL05163,"BROOKS, MO",REP,2014,AL,H,05,I,C,C00464149,7610 FOXFIRE DR.,,HUNTSVILLE,AL,35802
2,H0AL06088,"COOKE, STANLEY KYLE",REP,2010,AL,H,06,C,N,C00464222,723 CHERRY BROOK ROAD,,KIMBERLY,AL,35091
3,H0AL07094,"HILLIARD, EARL FREDERICK JR",DEM,2010,AL,H,07,O,P,C00460410,PO BOX 12804,,BIRMINGHAM,AL,35202
4,H0AR01083,"CRAWFORD, ERIC ALAN RICK",REP,2014,AR,H,01,I,C,C00462374,34 CR 455,,JONESBORO,AR,72404


In [19]:
# check out state abbreviations
cand.CAND_ST.value_counts()

CA    259
TX    188
FL    166
NY    121
IL     99
NC     97
MI     93
       88
NJ     82
OH     81
PA     78
VA     78
TN     73
GA     70
AZ     69
IN     68
WA     60
MD     58
MO     56
MA     51
CO     49
LA     47
SC     42
MN     40
NV     40
OR     39
UT     37
OK     37
WI     36
KY     35
CT     33
IA     31
AL     30
WV     29
AR     26
MS     25
NH     25
HI     23
KS     23
NE     23
NM     19
DC     18
ID     16
ME     14
MT     13
DE     13
AK     12
WY     11
RI      9
SD      7
VT      7
ND      6
PR      3
VI      3
ZZ      3
MP      3
AS      3
GU      2
AP      1
Name: CAND_ST, dtype: int64

In [274]:
# Check out party affiliation
cand.CAND_PTY_AFFILIATION.value_counts()
main = pd.Series(['REP','DEM','IND'])
len(cand[cand.CAND_PTY_AFFILIATION.isin(main)==False])/len(cand)
cand[cand.CAND_PTY_AFFILIATION.isin(main)==False].CAND_PTY_AFFILIATION.value_counts()

LIB    132
OTH     77
NNE     53
GRE     50
UNK     47
DFL      9
W        5
CON      5
IAP      5
CST      4
REF      4
         3
NPA      3
PG       2
IGR      2
AIP      2
IDP      1
IP       1
CIT      1
CRV      1
TEA      1
FED      1
HRP      1
NA       1
WFP      1
SUS      1
LPF      1
Name: CAND_PTY_AFFILIATION, dtype: int64

In [275]:
# Open text file containing committee dataset
df_list = []

with open(file='itpas2.txt',encoding='utf-8') as file:
    for row in file:
        df_list.append(file.readline()[:-1].split(sep="|"))

# Transform to Pandas data frame          
comm = pd.DataFrame(df_list)

# Add column headers from data dictionary
comm_cols = requests.get('http://classic.fec.gov/finance/disclosure/metadata/pas2_header_file.csv')
comm_cols = str(comm_cols.content[:-2]).split(sep=',')
comm_cols[0] = 'CMTE_ID'

comm.columns=comm_cols

# Visual inspection of data
comm.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID'
0,C00177436,N,M3,P,13961108067,24K,CCM,TIM SCOTT FOR SENATE,CHARLESTON,SC,...,,02182013,2500,C00540302,S4SC00240,35823277,858918,,,4030420131184959065
1,C00177436,N,M3,P,13961108068,24K,CCM,FRIENDS FOR HARRY REID,LAS VEGAS,NV,...,,02212013,2500,C00204370,S6NV00028,35831833,858918,,,4030420131184959067
2,C00177436,N,M3,P,13961108068,24K,CCM,RICHARD NEAL FOR CONGRESS COMMITTEE,SPRINGFIELD,MA,...,,02212013,1000,C00226522,H8MA02041,35831854,858918,,,4030420131184959069
3,C00177436,N,M3,P,13961108065,24K,CCM,FRIENDS OF MAX BAUCUS,HELENA,MT,...,,02132013,2500,C00328211,S8MT00010,35791855,858918,,,4030420131184959058
4,C00177436,N,M3,P,13961108065,24K,CCM,CANTOR FOR CONGRESS,RICHMOND,VA,...,,02132013,1000,C00355461,H0VA07042,35791857,858918,,,4030420131184959060


In [22]:
# Check out columns
comm.columns

Index(['CMTE_ID', 'AMNDT_IND', 'RPT_TP', 'TRANSACTION_PGI', 'IMAGE_NUM',
       'TRANSACTION_TP', 'ENTITY_TP', 'NAME', 'CITY', 'STATE', 'ZIP_CODE',
       'EMPLOYER', 'OCCUPATION', 'TRANSACTION_DT', 'TRANSACTION_AMT',
       'OTHER_ID', 'CAND_ID', 'TRAN_ID', 'FILE_NUM', 'MEMO_CD', 'MEMO_TEXT',
       'SUB_ID''],
      dtype='object')

In [276]:
# Examine featuers of comm dataset
comm.TRANSACTION_PGI.value_counts()

comm.MEMO_TEXT.value_counts()

comm.ENTITY_TP.value_counts()

comm[comm.ENTITY_TP==''].NAME.value_counts()

comm[abs(pd.to_numeric(comm.TRANSACTION_AMT)<0)].TRANSACTION_AMT
comm[pd.isnull(pd.to_numeric(comm.TRANSACTION_AMT))==True].TRANSACTION_AMT

sum(pd.to_numeric(comm[comm.ENTITY_TP==''].TRANSACTION_AMT))/sum(pd.to_numeric(comm[pd.isnull(comm.TRANSACTION_AMT)==False].TRANSACTION_AMT))*100

11.474192178469437

In [24]:
# checkout cand_id and see if there are duplicates, i.e. if there are differences in how cand_name is formatted

cand.CAND_ID.value_counts()
cand.CAND_NAME.value_counts()

KALEMKARIAN, TIMOTHY CHARLES       3
CARROLL, JERRY LEON                2
KENT, GAYLON                       2
BATES, DON JR                      2
KIRK, MARK STEVEN                  2
MCMILLAN, JAIME                    2
LEVENE, ALLAN                      2
ANGLE, SHARRON E                   2
MARTIN, ANDY                       2
ARMSTRONG, BRANDON CHRISTINA       2
SCHIESS, JOHN T                    2
SANCHEZ, DAVID                     2
KUCINICH, DENNIS J                 2
WEBB, JOHN                         2
POLIQUIN, BRUCE L                  2
SHAYS, CHRISTOPHER                 2
SCHONBERG, STEVE                   2
DAINES, STEVEN                     2
GOBERMAN, PAVEL                    2
SMITH, RHETT ROSENQUEST            2
BANKS, J E WENDELL KENNEDY         2
YU, EUGENE CHIN                    2
FULNER, ROBERT JAMES               2
EVANS, MERVIN LEON                 2
HOLCOMB, DAVID CHRISTOPHER         2
BERKLEY, SHELLEY                   2
OERTHER, BRIAN DOUGLAS             2
L

# Assess

* **Candidate dataset**
    * Rename confusing headings
        * CAND_ST is state, CAND_ST1 and CAND_ST2 are Street address √
        * CAND_ICI is race status (open vs. incumbent), CAND_STATUS is candidate status (incumbent vs. challenger) √
        * CAND_PCC means 'primary committee' √
    * Mixed data types
        * transform CAND_ID to category √
        * CAND_ELECTION_YR is type float √
    * Irrelevent data
        * Drop election years that are not 2014 √
    * Longtail party affiliations
        * Drop all parties except DEM, REP, IND, LIB, and GRE and the next biggest √
    * Columns we don't need
        * Remove all columns except CAND_ID, NAME, CAND_ELECTION_YR, CAND_PTY_AFFILIATION, CAND_STATE, CAND_RACE_STATUS, CAND_OFFICE √
    * State abbreviations outside states and territories 
        * ZZ √
        * blank √
    * Candidates with multiple candidate IDs (some explained by candidates running in multiple years) √


* **Committee dataset**
    * Confusing headings
        * (AMNDT_IND is amendment indicator, but we don't care about that)
        * TRANSACTION_PGI means primary or general election; change to "PRIMARY_OR_GENERAL" √
        * TRAN_ID means Transaction_ID √
    * Unclear values (the data, not the candidates)
        * TRANSACTION_PGI includes values sucha s "E," "S2014," "G", "G2014," "Pyyy," etc. 
            * bucket where comprehensible, drop where not √
        * TRANSACTION_AMT includes negative values and one null value √
            * 
        * 5,000 transactions for 1069 committees w/ blank entity type, totaling about 12% of contributions by $ value √
            * check if they are ever categorized; interpolate that categorization
            * if not, drop them √
    * Mixed data types
        * transform CMTE_ID to category √
        * transform CAND_ID to category √
        * transform TRANSACTION_AMT to float √
    * Columns we don't need
        * Remove all columns except CMTE_ID, TRANSACTION_AMOUNT, ENTITY_TYPE, CAND_ID, PAC_NAME, TRANSACTION_ID √
        * Different zip code formats (w/ and w/o postal code)
        * Missing state data
    * Transaction date is in a weird format (MMDDYYYY)

        

# Clean

#### Clean dirty data issues in Candidates dataset

In [277]:
# make a copy for cleaning
cand_clean = cand.copy()

# rename columns
cand_clean.rename(columns={'CAND_ST':'CAND_STATE',
                     'CAND_ICI':'CAND_RACE_STATUS',
                     'CAND_PCC':'CAND_PRIMARY_COMMITTEE',
                    'CAND_ST1':'CAND_STREET_ADDRESS',
                     'CAND_ST2':'CAND_STREET_ADDRESS_2',
                    'CAND_ELECTION_YR':'CAND_ELECTION_YEAR',
                    'CAND_PTY_AFFILIATION':'CAND_PARTY',
                          'CAND_OFFICE_DISTRICT':'CAND_DISTRICT'},inplace=True)

#recast column types
cand_clean.CAND_STATE = cand_clean.CAND_STATE.astype('category')
cand_clean.CAND_ID = cand_clean.CAND_ID.astype('category')
cand_clean.CAND_ELECTION_YEAR = pd.to_datetime(cand_clean.CAND_ELECTION_YEAR,format='%Y')

# remove data from non-major parties
cand_clean = cand_clean[cand_clean.CAND_PARTY.isin(['REP','DEM','IND','LIB','GRE'])]

# remove data from outside 50 states and territories
invalid_states = ['ZZ','',' ']
cand_clean = cand_clean[(cand_clean.CAND_STATE != 'ZZ') & (cand_clean.CAND_STATE != '')]

# examine and remove candidates with duplicate records if they are inaccurate
multi = cand_clean.CAND_NAME.value_counts().to_frame().reset_index()
multi.columns = ['CAND','COUNT']
cand_clean[cand_clean.CAND_NAME.isin(multi[multi.COUNT > 1].CAND)]

# there is a John Webb in MO in 2014 and a John Webb in CA in 2012. Drop CA John Webb, H2CA42122
# Eugene Chin Yu only ran for Senate, not House. Drop House Chin Yu, H4GA12051
# Allan Levene has actually run or attempted to run in 4 different states (!) Keep all records
cand_clean.drop(cand_clean.index[cand_clean.CAND_ID.isin(['H2CA42122','H4GA12051'])],axis=0,inplace=True)
cand_clean.CAND_NAME.value_counts()

#  Remove all columns except:
# CAND_ID, NAME, CAND_ELECTION_YR, CAND_PTY_AFFILIATION, CAND_STATE, CAND_RACE_STATUS, CAND_OFFICE
cand_cols = ['CAND_ID','CAND_NAME','CAND_ELECTION_YEAR','CAND_PARTY','CAND_STATE','CAND_DISTRICT','CAND_RACE_STATUS','CAND_OFFICE']
cand_merge = cand_clean[cand_cols]

# visual assessment
cand_merge.head(15)
cand_merge.tail(15)

,CAND_ID,CAND_NAME,CAND_ELECTION_YEAR,CAND_PARTY,CAND_STATE,CAND_DISTRICT,CAND_RACE_STATUS,CAND_OFFICE
2752,S8NJ00285,"SABRIN, MURRAY",2014-01-01,REP,NJ,00,C,S
2753,S8NJ00392,"ANDREWS, ROBERT E",2008-01-01,DEM,NJ,00,C,S
2754,S8NM00143,"HANNAN, JIM",2008-01-01,DEM,NM,00,O,S
2755,S8NM00184,"UDALL, TOM",2014-01-01,DEM,NM,00,I,S
2756,S8NV00073,"ENSIGN, JOHN E",2012-01-01,REP,NV,00,C,S
2757,S8OK00225,"WALLACE, STEPHEN P",2010-01-01,IND,OK,00,C,S
2758,S8OR00207,"MERKLEY, JEFFREY ALAN",2014-01-01,DEM,OR,00,I,S
2759,S8SD00024,"PRESSLER, LARRY",2014-01-01,IND,SD,00,O,S
2760,S8TN00253,"LEWIS, DANIEL T",2008-01-01,LIB,TN,00,C,S
2761,S8TX00244,"ROLAND, JON",2014-01-01,LIB,TX,00,C,S


#### Clean dirty data issues in committee dataset

In [288]:
# make a clean copy
comm_clean = comm.copy()

# rename columns
comm_clean.CAND_ID = comm.CAND_ID.astype('category')
comm_clean.CMTE_ID = comm_clean.CMTE_ID.astype('category')
comm_clean.rename(columns={'TRANSACTION_PGI':'PRIMARY_OR_GENERAL',
                           'TRANSACTION_AMT':'TRANSACTION_AMOUNT',
                          'CMTE_ID':'COMMITTEE_ID',
                          'NAME':'PAC_NAME',
                          'TRAN_ID':'TRANSACTION_ID',
                          'TRANSACTION_DT':'TRANSACTION_DATE'},inplace=True)

# bucket unclear values; recast data types; drop invalid data
comm_clean = comm_clean[comm_clean.PRIMARY_OR_GENERAL.isin(['G','G2014'])]
comm_clean.TRANSACTION_AMOUNT = pd.to_numeric(comm_clean.TRANSACTION_AMOUNT)
comm_clean = comm_clean[(comm_clean.TRANSACTION_AMOUNT > 0) & (comm_clean.TRANSACTION_AMOUNT.isnull()==False)]

# 5,000 transactions for 1069 committees have a blank for entity type, totaling about 12% of contributions by \$ value
# check if they are ever categorized; interpolate that categorization
# if not, drop them
names = pd.unique(comm_clean[comm_clean.ENTITY_TP==''].PAC_NAME)
mapper = []
for n in names:
    entity_list = pd.unique(comm_clean[comm_clean.PAC_NAME==n].ENTITY_TP)
    entity_list = entity_list[entity_list!='']
    if len(entity_list) > 0:
        mapper.append({n:entity_list})

# basically, the type of contribution depends on the transaction, as well as the committee. 
# Drop these rather than trying to interpolate
comm_clean = comm_clean[comm_clean.ENTITY_TP!='']

# Change transaction dates to datetime object, drop transaction dates past 2014
def my_datetime(arg):
    return pd.to_datetime(arg,format='%m%d%Y')

comm_clean.TRANSACTION_DATE = comm_clean[comm_clean.TRANSACTION_DATE!=''].TRANSACTION_DATE.apply(my_datetime)

# CMTE_ID, TRANSACTION_AMOUNT, ENTITY_TYPE, CAND_ID, PAC_NAME
comm_cols = ['COMMITTEE_ID','TRANSACTION_AMOUNT','CAND_ID','PAC_NAME','TRANSACTION_DATE']
comm_merge = comm_clean[comm_cols]

comm_merge.head()

,COMMITTEE_ID,TRANSACTION_AMOUNT,CAND_ID,PAC_NAME,TRANSACTION_DATE
59,C00430884,1000,H2TX05121,FRIENDS OF JEB HENSARLING,2013-03-07
60,C00280222,1500,S8AK00090,ALASKANS FOR BEGICH 2014,2013-03-29
71,C00280222,1000,S8MT00010,FRIENDS OF MAX BAUCUS,2013-03-29
81,C00280222,1500,S4MI00355,PETERS FOR CONGRESS,2013-03-29
115,C00388132,1000,S4PA00121,FRIENDS OF PAT TOOMEY,2013-02-25


# Merge datasets

In [289]:
# join committee to candidate on CAND_ID
# a left join gives us the most complete dataset;
# keeping only the intersection (inner join) cuts the dataset in half
# however, the reasons for missing candidate ids are not systematic.
# for the sake of this analysis, the smaller dataset is sufficient.

dat = pd.merge(comm_merge,cand_merge,on='CAND_ID',how='inner')

# remove data from years other than 2014
dat = dat[dat.CAND_ELECTION_YEAR=='2014']

dat.columns

Index(['COMMITTEE_ID', 'TRANSACTION_AMOUNT', 'CAND_ID', 'PAC_NAME',
       'TRANSACTION_DATE', 'CAND_NAME', 'CAND_ELECTION_YEAR', 'CAND_PARTY',
       'CAND_STATE', 'CAND_DISTRICT', 'CAND_RACE_STATUS', 'CAND_OFFICE'],
      dtype='object')

# Assess
Test that merge did not muck up any data

In [280]:
dat.tail()

,COMMITTEE_ID,TRANSACTION_AMOUNT,CAND_ID,PAC_NAME,TRANSACTION_DATE,CAND_NAME,CAND_ELECTION_YEAR,CAND_PARTY,CAND_STATE,CAND_DISTRICT,CAND_RACE_STATUS,CAND_OFFICE
40903,C90011677,195000,S2NC00257,"CRAFT MEDIA/DIGITAL, LLC",2014-10-28,"HAUGH, SEAN",2014-01-01,LIB,NC,00,C,S
40904,C90011800,250,H6IL05150,VERVEMAIL LLC,2014-11-04,"KOLBER, VINCENT A",2014-01-01,REP,IL,05,C,H
40905,C00271338,5000,H2PA13078,MARJORIE 2014,2014-02-06,"MARGOLIES-MEZVINSKY, MARJORIE",2014-01-01,DEM,PA,13,O,H
40906,C00016899,1200,H4OH16115,CROSSLAND FOR CONGRESS,2014-12-01,"CROSSLAND, PETE",2014-01-01,DEM,OH,16,C,H
40907,C00345199,1000,H4CA07055,BIRMAN FOR CONGRESS,2014-04-01,"BIRMAN, IGOR A",2014-01-01,REP,CA,07,C,H


In [281]:
print(len(comm_merge),len(dat))

81043 40185


In [282]:
# Check for NaN values in CAND_NAME
print('number of contributions with missing candidate name: ',len(dat[dat.CAND_NAME.isnull()==True]))
print('number of candidate ids for contributions with missing candidate name: ',len(pd.unique(dat[dat.CAND_NAME.isnull()==True].CAND_ID)))

number of contributions with missing candidate name:  0
number of candidate ids for contributions with missing candidate name:  0


In [57]:
'''Before choosing to keep only the intersection of the two datasets, I investigated the reasons for missing CAND_ID

# Who are these missing Candidates?

missing.sample(5)

    # campaing directors
    # folks who did not actually run (i.e. the money was raised but the person did not stand)
    # people who are inexplicably missing. E.g., why do we have Justin Jelincic, who lost the CA-13 Dem primary
    # but not Barbara Lee who won the district in the general election?

cand_clean[(cand_clean.CAND_STATE=='CA') & (cand_clean.CAND_DISTRICT=='13')]    

cand_clean[(cand_clean.CAND_STATE=='TX') & (cand_clean.CAND_DISTRICT=='25')]

sample_cands = pd.Series(pd.unique(db.CAND_ID)).sample(20)
sample = db[db.CAND_ID.isin(sample_cands)]

for i in sample:
    comm_reference = comm[comm.TRANSACTION_ID==sample.TRANSACTION_ID[i]]
    cand_reference = cand[cand.CAND_ID==sample.CAND_ID[i]]
    assert comm_reference.OCCUPATION == sample.OCCUPATION[i],'Merge create inaccurate committee data'
    assert cand_reference.CAND_ZIP == sample.CAND_ZIP[i], 'Merge created inaccurate candidate data' '''

"Before choosing to keep only the intersection of the two datasets, I investigated the reasons for missing CAND_ID\n\n# Who are these missing Candidates?\n\nmissing.sample(5)\n\n    # campaing directors\n    # folks who did not actually run (i.e. the money was raised but the person did not stand)\n    # people who are inexplicably missing. E.g., why do we have Justin Jelincic, who lost the CA-13 Dem primary\n    # but not Barbara Lee who won the district in the general election?\n\ncand_clean[(cand_clean.CAND_STATE=='CA') & (cand_clean.CAND_DISTRICT=='13')]    \n\ncand_clean[(cand_clean.CAND_STATE=='TX') & (cand_clean.CAND_DISTRICT=='25')]\n\nsample_cands = pd.Series(pd.unique(db.CAND_ID)).sample(20)\nsample = db[db.CAND_ID.isin(sample_cands)]\n\nfor i in sample:\n    comm_reference = comm[comm.TRANSACTION_ID==sample.TRANSACTION_ID[i]]\n    cand_reference = cand[cand.CAND_ID==sample.CAND_ID[i]]\n    assert comm_reference.OCCUPATION == sample.OCCUPATION[i],'Merge create inaccurate commi

#### Assessing Tidiness

In [283]:
tidy_test = dat.groupby(by=['COMMITTEE_ID','CAND_ID','TRANSACTION_DATE','TRANSACTION_AMOUNT']).size().sort_values(ascending=False)
tidy_test.head(20)

COMMITTEE_ID  CAND_ID    TRANSACTION_DATE  TRANSACTION_AMOUNT
C90011156     S4MI00355  2014-09-23        55                    30
                         2014-09-16        55                    27
                         2014-09-26        55                    25
                         2014-09-18        55                    24
                         2014-10-20        55                    24
                         2014-10-03        55                    24
                         2014-10-13        55                    23
                         2014-09-25        55                    23
                         2014-10-07        55                    23
                         2014-10-29        55                    23
                         2014-10-01        55                    22
                         2014-09-29        55                    22
                         2014-10-21        55                    22
                         2014-09-30        55         

- **Conclusion:** merged dataframe is not tidy. Some committees have made multiple contributions of the same amount of money to the same candidate on the same date. In the original dataframe, each transaction had its own id; however, since there's no analysis we want to perform using those ids, this seems irrelevant.


- Next steps:
    - First, check that this is the case in the original committee contributions dataframe. Could the merge have created duplicate data I did not notice?
    - To tidy, we can sum up total contribution by committee, candidate id, and date. Each row will then represent a single day's worth of data per committee per candidate--so each observation will have its own row (and each variable already has its own column), meeting the tidiness requirements. 
    - We may want to aggregate again later in analysis--e.g. we can look at contributions by committee overall--but for now let's leave it at this level of granularity.

In [290]:
comm_merge.head()

,COMMITTEE_ID,TRANSACTION_AMOUNT,CAND_ID,PAC_NAME,TRANSACTION_DATE
59,C00430884,1000,H2TX05121,FRIENDS OF JEB HENSARLING,2013-03-07
60,C00280222,1500,S8AK00090,ALASKANS FOR BEGICH 2014,2013-03-29
71,C00280222,1000,S8MT00010,FRIENDS OF MAX BAUCUS,2013-03-29
81,C00280222,1500,S4MI00355,PETERS FOR CONGRESS,2013-03-29
115,C00388132,1000,S4PA00121,FRIENDS OF PAT TOOMEY,2013-02-25


In [311]:
# loop through a bunch of committee ids
# check that the original dataframe always has more repeated transactions than the merged dataframe;
# our merged dataset is a subset of committee contributions, so it should never have MORE repeated 
# transactions for a given committee/candidate/date combination than the original comm dataset

for i in range(0,200):
    random_id = (dat.COMMITTEE_ID.unique()[random.randint(0,len(dat.COMMITTEE_ID.unique()))])
    committee_test = comm_merge[comm_merge.COMMITTEE_ID.isin([random_id])].groupby(by=['COMMITTEE_ID','CAND_ID','TRANSACTION_DATE']).size()
    dat_test = dat[dat.COMMITTEE_ID.isin([random_id])].groupby(by=['COMMITTEE_ID','CAND_ID','TRANSACTION_DATE']).size()
    assert len(dat_test[dat_test>1]) <= len(committee_test[committee_test>1]), 'there are more observations in the merged dataframe than the original for id ' + random_id
    
# ok, it seems like we haven't created duplicates

In [312]:
# next, aggregate by committee id/candidate id/

COMMITTEE_ID  CAND_ID    TRANSACTION_DATE
C00388777     S2TX00106  2014-10-01          1
dtype: int64

In [327]:
# another quick check: do we ever have candidate id's with multiple ways of writing candidate id?
errors = []
for i in dat.CAND_ID.unique():
    x = dat[dat.CAND_ID==i].CAND_NAME.value_counts()
    if len(x) > 1:
        errors.append(i)
        
assert len(errors) == 0, "the following candidates have more than 1 way of recording their name for the same id: " + errors

In [ ]:
# since this is an R assignment, let's do the final group by/aggregation in R :)

In [329]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40185 entries, 0 to 40907
Data columns (total 12 columns):
COMMITTEE_ID          40185 non-null category
TRANSACTION_AMOUNT    40185 non-null int64
CAND_ID               40185 non-null object
PAC_NAME              40185 non-null object
TRANSACTION_DATE      40035 non-null datetime64[ns]
CAND_NAME             40185 non-null object
CAND_ELECTION_YEAR    40185 non-null datetime64[ns]
CAND_PARTY            40185 non-null object
CAND_STATE            40185 non-null category
CAND_DISTRICT         40185 non-null object
CAND_RACE_STATUS      40185 non-null object
CAND_OFFICE           40185 non-null object
dtypes: category(2), datetime64[ns](2), int64(1), object(7)
memory usage: 3.7+ MB


In [330]:
# write clean data to a csv to open in Rstudio
file_name = 'campaign_finance_clean_data.csv'
dat.to_csv(path_or_buf=file_name,sep=',')